Scopus (Elsevier and ScienceDirect) should be another good source of metadata for our research references. I need to keep working with the various API end points to come up with the right combination of queries and data structures, and some of these are going to have to be run from inside the USGS network. Scopus uses a combination of originating IP and a registered API key to authorize some of the APIs based on institution subscription.

Initially, this notebook just runs through the cases where we've already determined a DOI and grabs a basic Scopus record for future reference.

In [1]:
import requests
from IPython.display import display
from datetime import datetime
from bis import dd
from bis import rrl

In [2]:
bis = dd.getDB("bis")
collection_rrl = bis["RRL"]

In [ ]:
count = 0
for record in collection_rrl.find({"$and":[{"$or":[{"CrossRef.Record.DOI":{"$exists":True}},{"Link Metadata.Link Response.DOI":{"$exists":True}}]},{"Scopus.ScopusRecord":{"$exists":False}}]}):
    try:
        recordDOI = record["Link Metadata"]["Link Response"]["DOI"]
    except:
        recordDOI = record["CrossRef"]["Record"]["DOI"]
    
    scopusSearch = rrl.ResearchReferenceLibrary.lookup_scopus_by_doi(recordDOI)
    scopusRecord = {}
    scopusRecord["Date Retrieved"] = datetime.utcnow().isoformat()
    scopusRecord["Search Term"] = "doi"
    scopusRecord["Search Value"] = recordDOI

    if scopusSearch["search-results"]["opensearch:totalResults"] == "1":
        scopusRecord["Cached Result"] = scopusSearch["search-results"]["entry"][0]
    else:
        scopusRecord["Cached Result"] = False

    print (count, collection_rrl.update_one({"_id":record["_id"]},{"$set":{"Scopus.ScopusRecord":scopusRecord}}))
    
    count = count + 1
    

0 <pymongo.results.UpdateResult object at 0x10763a6c0>
1 <pymongo.results.UpdateResult object at 0x106eb13f0>
2 <pymongo.results.UpdateResult object at 0x107657e10>
3 <pymongo.results.UpdateResult object at 0x1074c8318>
4 <pymongo.results.UpdateResult object at 0x1059c3e58>
5 <pymongo.results.UpdateResult object at 0x106ef3af8>
6 <pymongo.results.UpdateResult object at 0x106eb13f0>
7 <pymongo.results.UpdateResult object at 0x107657e10>
8 <pymongo.results.UpdateResult object at 0x10763a048>
9 <pymongo.results.UpdateResult object at 0x106d9d990>
10 <pymongo.results.UpdateResult object at 0x107331048>
11 <pymongo.results.UpdateResult object at 0x106ea0dc8>
12 <pymongo.results.UpdateResult object at 0x107657e10>
13 <pymongo.results.UpdateResult object at 0x106ea00d8>
14 <pymongo.results.UpdateResult object at 0x10763a048>
15 <pymongo.results.UpdateResult object at 0x107331048>
16 <pymongo.results.UpdateResult object at 0x106eb13f0>
17 <pymongo.results.UpdateResult object at 0x107331048>
18

# Next Steps
The next thing I'll try once I get back into the office is to grab the citation summary for each of these and work with the search APIs to see where I can fill in blanks. Scopus search will be a little more difficult than CrossRef search in that I need to get a reasonable parsing of the citation metadata to use its parts instead of just sending along the messy string. So, I'll need to revisit the parsing mechanisms again to see how good I can make those.